# Network topology for simulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
from iddn_paper.old_functions import sim3_network_topo as nett
from iddn_paper import sim3_h5op, tool_sys

top_folder = tool_sys.get_work_folder() + "/experiment_iddn_paper/"


In [ ]:
def shift_net(gene_type, gene_par, n_shift):
    gene_type_shift = dict()
    gene_par_shift = dict()
    for key in gene_type.keys():
        gene_type_shift[key+n_shift] = gene_type[key]
        par_shift = []
        for par in gene_par[key]:
            par_shift.append(par+n_shift)
        gene_par_shift[key+n_shift] = par_shift
    return gene_type_shift, gene_par_shift


In [ ]:
n_net = 2
n_hub_regulator = 1
n_secondary_regulator = 15
n_other_genes = 10
two_condition_ratio = 0.25
n_sample_gen = 10000

reload(nett)
gene_type = dict()
gene_par = dict()
for n in range(n_net):
    gene_type0, gene_par0, regulator_weight0 = nett.create_gene_network(
        n_hub_regulator=n_hub_regulator,
        n_secondary_regulator=n_secondary_regulator,
        n_other_genes=n_other_genes,
        wt_top_regulator=1,
    )
    gene_type_shift, gene_par_shift = shift_net(gene_type0, gene_par0, len(gene_type))
    gene_type.update(gene_type_shift)
    gene_par.update(gene_par_shift)


In [ ]:
mol_layer, mol_par, mol_par_roles, mol_type = nett.create_mol_network(gene_type, gene_par)
idx_layer, idx_par, idx_par_roles, mol2idx, idx2mol, layer_cnt = nett.mol_network_to_index(mol_layer, mol_par, mol_par_roles)
net_info, dep_mat, con_mat = nett.prep_net_for_sim(
    mol_layer,
    mol_par,
    mol_par_roles,
    mol_type=None,
)

dep_mat_null = np.ones_like(dep_mat)
reload(nett)
dep_mat_prior, dep_mat_prior_loose = nett.make_iddn_dep_prior(mol_type, net_info["mol2idx"])

idx = np.random.randint(1000000)
exp_name = (
    f"sim3_tf_mrna_mirna_n_{len(con_mat)}"
    f"_ratio_{two_condition_ratio}_{idx}"
)

In [ ]:
import networkx as nx
G1 = nx.from_numpy_array(con_mat)
nx.draw_networkx(G1, node_size=10, with_labels=False)

## Steady state simulation

In [ ]:
if 0:
    mol_par1, mol_par2, mol_par_roles1, mol_par_roles2 = nett.make_two_conditions_mol_net(
        mol_par,
        mol_par_roles,
        ratio=two_condition_ratio,
    )

    net_info1, dep_mat1, con_mat1 = nett.prep_net_for_sim(
        mol_layer,
        mol_par1,
        mol_par_roles1,
        mol_type=None,
    )
    net_info2, dep_mat2, con_mat2 = nett.prep_net_for_sim(
        mol_layer,
        mol_par2,
        mol_par_roles2,
        mol_type=None,
    )
    
    comm_gt, diff_gt = tools.get_common_diff_net_topo([con_mat1, con_mat2])
    sigma_mid_vec = np.zeros(len(mol_layer)) + 2.0
    # sigma_mid_vec = np.random.uniform(low=0.5, high=2.5, size=len(mol_layer))
    
    sigma_in = 2.0
    hill_coef = 1.0
    
    dat1, state_history1, noise_history1 = sim3_steady_state_batch.run_sim(
        net_info1["idx_layer"],
        net_info1["idx_par"],
        net_info1["idx_par_roles"],
        n_sample=n_sample_gen,
        n_max_steps=100,
        method="steady",
        sigma_in=sigma_in,
        sigma_mid=sigma_mid_vec,
        hill_coef=hill_coef,
    )
    dat2, state_history2, noise_history2 = sim3_steady_state_batch.run_sim(
        net_info2["idx_layer"],
        net_info2["idx_par"],
        net_info2["idx_par_roles"],
        n_sample=n_sample_gen,
        n_max_steps=100,
        method="steady",
        sigma_in=sigma_in,
        sigma_mid=sigma_mid_vec,
        hill_coef=hill_coef,
    )

In [ ]:
reload(sim3_h5op)
if 0:
    dat_file = f"{top_folder}/sim_input/{exp_name}.hdf5"
    sim3_h5op.make_new_sim_data(
        dat_file,
        dat1,
        dat2,
        con_mat1,
        con_mat2,
        comm_gt=comm_gt,
        diff_gt=diff_gt,
        dep_mat_null=dep_mat_null,
        dep_mat_prior=dep_mat_prior,
        dep_mat_prior_loose=dep_mat_prior,
        mol_names=list(idx2mol.values()),  # names with same order as data
        layer_count=net_info1["layer_count"],
    )
    print(exp_name)

## GGM simulation

In [ ]:
# plt.imshow(con_mat1)

In [ ]:
# cc1 = np.corrcoef(dat1.T)
# omega_org1 = np.linalg.inv(cc1)
# omega1 = np.copy(omega_org1)
# omega1[np.abs(omega_org1) < 0.1] = 0
# plt.imshow(np.abs(omega1)>0)

In [ ]:
# omega1_x = np.copy(omega1)
# omega1_x[np.arange(len(omega1_x)), np.arange(len(omega1_x))] = 0
# plt.imshow(np.abs(omega1_x))

In [ ]:
# cc2 = np.corrcoef(dat2.T)
# omega_org2 = np.linalg.inv(cc2)
# omega2 = np.copy(omega_org2)
# omega2[np.abs(omega_org2) < 0.1] = 0

In [ ]:
from ddn3 import simulation
from ddn3_extra import simulation_r

# def mod_omega(omega):
#     # omega = np.copy(con_mat)
#     xx = np.maximum(np.sum(omega, axis=0)[:] + np.sum(omega, axis=1)[:], 1.0)
#     omega[np.arange(len(omega)), np.arange(len(omega))] = xx
#     eigx, _ = np.linalg.eig(omega)
#     print(np.min(np.real(eigx)))
#     # omega, _ = simulation_r.make_two_from_one(
#     #     omega, ratio_diff=0, verbose=True)
#     return omega
# 
# omega1 = mod_omega(1*(con_mat1>0))
# omega2 = mod_omega(1*(con_mat2>0))

omega = np.copy(1*(con_mat>0))
xx = np.maximum(np.sum(omega, axis=0)[:] + np.sum(omega, axis=1)[:], 1.0)
omega[np.arange(len(omega)), np.arange(len(omega))] = xx

dep_in = dep_mat_prior + dep_mat_prior.T
reload(simulation_r)
omega1, omega2 = simulation_r.make_two_from_one(omega, dep_in, ratio_diff=0.25, verbose=True)


In [ ]:
cov1x, omega1x = simulation.create_cov_prec_mat(omega1)
cov2x, omega2x = simulation.create_cov_prec_mat(omega2)
g1_cov, g2_cov, comm_gt_g, diff_gt_g = simulation.prep_sim_from_two_omega(omega1, omega2)
con_mat1 = 1 * (np.abs(omega1) > 1e-8)
con_mat2 = 1 * (np.abs(omega2) > 1e-8)
n_node = len(omega1)
con_mat1[np.arange(n_node), np.arange(n_node)] = 0
con_mat2[np.arange(n_node), np.arange(n_node)] = 0

dat1_g, dat2_g = simulation.gen_sample_two_conditions(g1_cov, g2_cov, n_sample_gen, n_sample_gen)


In [ ]:
f_out = f"{exp_name}_ggm"
dat_file_g = f"{top_folder}/sim_input/{f_out}.hdf5"

if 1:
    sim3_h5op.make_new_sim_data(
        dat_file_g,
        dat1_g,
        dat2_g,
        con_mat1,
        con_mat2,
        comm_gt=comm_gt_g,
        diff_gt=diff_gt_g,
        dep_mat_null=dep_mat_null,
        dep_mat_prior=dep_mat_prior,
        dep_mat_prior_loose=dep_mat_prior_loose,
        mol_names=list(idx2mol.values()),
        layer_count=layer_cnt,
    )
    print(f_out)


In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15, 6))
print(np.mean(np.abs(g1_cov)))
print(np.mean(np.abs(g2_cov)))

# im0 = ax[0].imshow(omega1, cmap="bwr")
# im1 = ax[1].imshow(omega2, cmap="bwr")
im0 = ax[0].imshow(g1_cov, cmap="bwr")
im1 = ax[1].imshow(g2_cov, cmap="bwr")
# im0.set_clim(-0.4,0.4)
# im1.set_clim(-0.4,0.4)
im0.set_clim(-1,1)
im1.set_clim(-1,1)
fig.colorbar(im0, ax=ax[0])
fig.colorbar(im1, ax=ax[1])

## Debug

In [ ]:
# plt.imshow(dep_mat_prior)